In [1]:
import transformers 
import datasets
import torch
import logging

c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# squad = load_dataset("squad", split="train[:5000]")

# squad = datasets.load_dataset("squad")

Found cached dataset squad (C:/Users/dama_/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 199.96it/s]


In [2]:
squad=torch.load("../datasets/squad.pt")

In [3]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [33]:
# squad = squad.train_test_split(test_size=0.2)

In [4]:
# Preprocessing
# sep_token = '<sep>'
dataset_name = "squad"
model_type="bert"
model_name= "distilbert-base-uncased"
models_dir = "saved_models/distilbert-base-uncased_mod"
checkpoint = 'distilbert-base-uncased'
max_input_length = 384

# ## Training
learning_rate = 3e-5


In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [5]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_input_length ,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [6]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Loading cached processed dataset at C:\Users\dama_\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-c6979b8502e8782b.arrow
Loading cached processed dataset at C:\Users\dama_\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-5208e4cd58802a97.arrow


In [7]:
data_collator = transformers.DefaultDataCollator()

In [8]:
model = transformers.AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [9]:
training_args = transformers.TrainingArguments(
    output_dir=models_dir,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)



e:\HOGENT\2022_2023\BA\BP_Info_Support\bert\saved_models/distilbert-base-uncased_mod is already a clone of https://huggingface.co/damapika/distilbert-base-uncased_mod. Make sure you pull the latest changes with `repo.git_pull()`.


In [10]:
trainer.train()

c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: damapika. Use `wandb login --relogin` to force relogin


  3%|▎         | 500/16425 [03:58<2:05:34,  2.11it/s]

{'loss': 2.721, 'learning_rate': 2.908675799086758e-05, 'epoch': 0.09}


  6%|▌         | 1000/16425 [08:10<2:02:15,  2.10it/s]

{'loss': 1.6817, 'learning_rate': 2.8173515981735162e-05, 'epoch': 0.18}


...

 97%|█████████▋| 16000/16425 [2:15:02<03:15,  2.18it/s]  

{'loss': 0.6164, 'learning_rate': 7.76255707762557e-07, 'epoch': 2.92}


                                                       
100%|██████████| 16425/16425 [2:20:08<00:00,  1.95it/s]

{'eval_loss': 1.1685752868652344, 'eval_runtime': 94.783, 'eval_samples_per_second': 111.518, 'eval_steps_per_second': 6.974, 'epoch': 3.0}
{'train_runtime': 8410.9413, 'train_samples_per_second': 31.245, 'train_steps_per_second': 1.953, 'train_loss': 0.9905400338601122, 'epoch': 3.0}


TrainOutput(global_step=16425, training_loss=0.9905400338601122, metrics={'train_runtime': 8410.9413, 'train_samples_per_second': 31.245, 'train_steps_per_second': 1.953, 'train_loss': 0.9905400338601122, 'epoch': 3.0})

In [11]:
trainer.push_to_hub()

Upload file pytorch_model.bin: 259MB [00:39, 7.26MB/s]                            To https://huggingface.co/damapika/distilbert-base-uncased_mod
   46e0351..1a9dcfe  main -> main

Upload file pytorch_model.bin: 100%|██████████| 253M/253M [00:40<00:00, 6.63MB/s]
Upload file runs/Apr21_22-30-03_Damapika/events.out.tfevents.1682109011.Damapika.33640.0: 100%|██████████| 9.89k/9.89k [00:40<00:00, 253B/s] 
To https://huggingface.co/damapika/distilbert-base-uncased_mod
   1a9dcfe..f9707b1  main -> main



'https://huggingface.co/damapika/distilbert-base-uncased_mod/commit/1a9dcfe661042d1eb582247b0b8cc042eb868153'

In [7]:
question = "Who is Fyodor Dostoevsky?"
context = "In the world of literature, there have been many authors who have gained a reputation for their ability to create complex characters. One such author is Fyodor Dostoevsky, a Russian novelist who wrote several influential works in the 19th century."

In [8]:
question_answerer = transformers.pipeline("question-answering", model="damapika/distilbert-base-uncased_mod")
question_answerer(question=question, context=context)

{'score': 0.6729146242141724,
 'start': 172,
 'end': 190,
 'answer': 'a Russian novelist'}

In [24]:
#optional  manually replicate the results of the pipeline 

tokenizer = transformers.AutoTokenizer.from_pretrained("damapika/distilbert-base-uncased_mod")
inputs = tokenizer(question, context, return_tensors="pt")

In [25]:


model = transformers.AutoModelForQuestionAnswering.from_pretrained("damapika/distilbert-base-uncased_mod")
with torch.no_grad():
    outputs = model(**inputs)

In [26]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [27]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'a russian novelist'